In [1]:
# p-Werte für zwei Geräte Vergleich mit McNemar

from statsmodels.stats.contingency_tables import mcnemar
import pandas as pd

# Dateninput
data = pd.DataFrame({
    "Dilution": ["Undiluted", "1:3", "1:9", "1:27", "1:81", "1:243", "1:729", "1:2187", "1:6561"],
    "Turbo_TSI": [100, 100, 96, 80, 53, 28, 9, 5, 0],
    "Immulite": [99, 97, 87, 57, 27, 3, 0, 0, 0],
    "Cobas": [100, 79, 37, 10, 0, 0, 0, 0, 0],
    "Kryptor": [100, 76, 37, 14, 0, 0, 0, 0, 0]
})

# McNemar-Test Funktion  mit statsmodels
def perform_mcnemar_statsmodels(data, device_names, n_samples):
    results = []

    for device in device_names:
        for index, row in data.iterrows():
            dilution = row["Dilution"]
            turbo_positive = row["Turbo_TSI"]
            device_positive = row[device]

            # Kontingenztafel erstellen, leichte Annäherung
            both_positive = min(turbo_positive, device_positive)
            only_turbo = turbo_positive - both_positive
            only_device = device_positive - both_positive
            both_negative = n_samples - turbo_positive - only_device

            table = [[both_positive, only_turbo], [only_device, both_negative]]

            # Exakter McNemar-Test aus table
            result = mcnemar(table, exact=True)
            results.append({
                "Dilution": dilution,
                "Device": device,
                "p_value": result.pvalue
            })

    return results

# Geräte eingabe und McNemar Test ausführen
devices = ["Immulite", "Cobas", "Kryptor"]
results = perform_mcnemar_statsmodels(data, devices, n_samples=100)

# Tabellenüberschrift + Trennlinie + Ergebnisse
print(f"{'Verdünnung':<12} {'Gerät':<10} {'p-Wert':>12}")
print("-" * 34)

for result in results:
    print(f"{result['Dilution']:<12} {result['Device']:<10} {result['p_value']:>12.10f}")

# veraltete Ausgabe
#for result in results:
#    print(f"Verdünnung: {result['Dilution']}, Gerät: {result['Device']}, p-Wert: {result['p_value']:.10f}")


Verdünnung   Gerät            p-Wert
----------------------------------
Undiluted    Immulite   1.0000000000
1:3          Immulite   0.2500000000
1:9          Immulite   0.0039062500
1:27         Immulite   0.0000002384
1:81         Immulite   0.0000000298
1:243        Immulite   0.0000000596
1:729        Immulite   0.0039062500
1:2187       Immulite   0.0625000000
1:6561       Immulite   1.0000000000
Undiluted    Cobas      1.0000000000
1:3          Cobas      0.0000009537
1:9          Cobas      0.0000000000
1:27         Cobas      0.0000000000
1:81         Cobas      0.0000000000
1:243        Cobas      0.0000000075
1:729        Cobas      0.0039062500
1:2187       Cobas      0.0625000000
1:6561       Cobas      1.0000000000
Undiluted    Kryptor    1.0000000000
1:3          Kryptor    0.0000001192
1:9          Kryptor    0.0000000000
1:27         Kryptor    0.0000000000
1:81         Kryptor    0.0000000000
1:243        Kryptor    0.0000000075
1:729        Kryptor    0.0039062500
1:2

In [39]:
# p-Werte für Redilutions mit McNemar

import numpy as np
from statsmodels.stats.contingency_tables import mcnemar

# Daten
n = 100
turbo_positive = 44      # Pos. Samples Turbo
device_positive = 42    # Pos. Samples Immulite-Kryptor-Cobas

# Kontingenztabelle + McNemar
both_positive = min(turbo_positive, device_positive)
only_turbo = turbo_positive - both_positive
only_device = device_positive - both_positive
both_negative = n - turbo_positive - only_device

table = np.array([[both_positive, only_turbo], [only_device, both_negative]])
result = mcnemar(table, exact=True)

# Output
print("\nMcNemar test statistic:", result.statistic)
print(f"p-value: {result.pvalue:.10f}")


McNemar test statistic: 0.0
p-value: 0.5000000000


In [5]:
from scipy.stats import chi2_contingency

# Kontingenztabellen (positiv und negativ für 5 Geräte)
datasets = [
    [  # Undiluted
        [100, 0],  # TurboTSI
        [99, 1],  # Immulite
        [100, 0],  # Cobas
        [100, 0],  # Kryptor
    ],
    [  # 1:3
        [100, 0],  # TurboTSI
        [97, 3],  # Immulite
        [79, 21],  # Cobas
        [76, 24],  # Kryptor
    ],
    [  # 1:9
        [96, 4],  # TurboTSI
        [87, 13],  # Immulite
        [37, 63],  # Cobas
        [37, 63],  # Kryptor
    ],
    [  # 1:27
        [80, 20],  # TurboTSI
        [57, 43],  # Immulite
        [10, 90],  # Cobas
        [14, 96],  # Kryptor
    ],
    [  # 1:81
        [53, 47],  # TurboTSI
        [27, 73],  # Immulite
        [0, 100],  # Cobas
        [0, 100],  # Kryptor
    ]
]

for idx, data in enumerate(datasets):
    print(f"Verdünnung {idx}:")
    chi2, p, dof, expected = chi2_contingency(data)
    print(f"  Chi-Quadrat-Statistik: {chi2}")
    print(f"  p-Wert: {p}")
    print(f"  Freiheitsgrade: {dof}")
    # print(f"  Erwartete Häufigkeiten:\n{expected}\n")


Verdünnung 0:
  Chi-Quadrat-Statistik: 3.0075187969924815
  p-Wert: 0.3904673777581044
  Freiheitsgrade: 3
Verdünnung 1:
  Chi-Quadrat-Statistik: 42.61363636363636
  p-Wert: 2.972593691962033e-09
  Freiheitsgrade: 3
Verdünnung 2:
  Chi-Quadrat-Statistik: 131.07670539577154
  p-Wert: 3.1698161004309566e-28
  Freiheitsgrade: 3
Verdünnung 3:
  Chi-Quadrat-Statistik: 151.16322772739744
  p-Wert: 1.4785306850448233e-32
  Freiheitsgrade: 3
Verdünnung 4:
  Chi-Quadrat-Statistik: 121.125
  p-Wert: 4.4171232903792865e-26
  Freiheitsgrade: 3
